# NEXUS tool: case study for the NWSAS basin - least-cost options calculations
In this notebook a case study for the NWSAS basin is covered using the `nexus_tool` package. The energy requirements for agricultural irrigation were previously calculated (check the water demand and energy demand case studies). In this case study, the least-cost options to supply the energy needs for agricultural irrigation are identified.

First import the package by running the following block:

In [66]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
%autoreload
import nexus_tool

[autoreload of nexus_tool failed: Traceback (most recent call last):
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 387, in update_generic
    update(a, b)
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 342, in update_class
    if update_generic(old_obj, new_obj): continue
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 387, in update_generic
    update(a, b)
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packa

In [68]:
file_path = r'nwsas_results/pumping_energy.csv'
df = nexus_tool.read_csv(file_path)

In [69]:
file_path = r'nwsas_data/Fuel_prices.xlsx'
fuel_price = nexus_tool.read_excel(file_path)

In [70]:
nwsas = nexus_tool.Model(df)
nwsas.end_year = 2030
nwsas.start_year = 2016
nwsas.discount_rate = 0.05

In [71]:
nwsas.df = nwsas.df.merge(fuel_price, on='country')

In [72]:
nwsas.create_wind_turbine('Wind power', life=20,
                          om_cost=0.02, capital_cost=1300,
                          efficiency=0.6)

In [73]:
nwsas.create_pv_system('Solar PV', life=15,
                       om_cost=0.01, capital_cost=1140,
                       efficiency=0.7)

In [74]:
nwsas.create_standard_tech('Diesel set', life=10, om_cost=0.1,
                           capital_cost=938, fuel_cost=nwsas.df['Dprice1'],
                           fuel_req=0.095, efficiency=0.27, cf = 0.5,
                           emission_factor=2.7, env_cost=0)

In [75]:
nwsas.create_standard_tech('Grid pump', life=10, om_cost=0.1,
                           capital_cost=845, fuel_cost=nwsas.df['Epricelow'],
                           fuel_req=1, efficiency=0.55, cf = 0.8,
                           emission_factor=0.728, env_cost=0)

In [76]:
nwsas.get_cf(['Wind power', 'Solar PV'])
nwsas.get_installed_capacity('all')
nwsas.get_max_capacity('a')
nwsas.get_lcoe()

In [77]:
comp_vars = {'Algeria':['Diesel set','Solar PV','Wind power'],
            'Tunisia':['Diesel set','Solar PV','Wind power'],
            'Libya':['Grid pump','Solar PV','Wind power']}
nwsas.get_least_cost(technologies = comp_vars, geo_boundary = 'country')

In [78]:
nwsas.get_pumping_cost(inplace = True)
nwsas.get_unit_pumping_cost(inplace=True)

In [85]:
nwsas.print_summary(['country','province'])

Irrigated area (ha)  Water demand (Mm3)  \
country province                                               
Algeria Adrar               24101.816848          380.659062   
        Biskra               5749.995666           57.693849   
        Djelfa               3449.972096           35.546265   
        El Oued             68999.985344          791.446126   
        Ghardaia            34499.998422          440.742945   
        Illizi               1770.396215           25.085808   
        Khenchela            1149.970213           12.454845   
        Laghouat             5749.994721           57.985985   
        Ouargla             32699.346634          404.789172   
        Tamanrasset          4868.857546           76.228504   
        Tebessa              4599.970966           50.265413   
Libya   Ghadamis             5353.213079           56.291559   
        Gharyan              8557.616421           92.582518   
        Jufrah               7963.713904          120.357521   
        Musrata             21082.919486          187.440532   
Tunisia Gabes                2874.983928           28.963020   
        Kebili              27412.511949          307.219625   
        Tataouine            1833.775565           18.733180   
        Tozeur               9617.446033          103.812085   

                     Water intensity (m3/ha)  Energy demand (GWh)  \
country province                                                    
Algeria Adrar                   15793.791148           117.426808   
        Biskra                  10033.720376            20.410051   
        Djelfa                  10303.348469             3.153813   
        El Oued                 11470.236142           200.007164   
        Ghardaia                12775.158413            83.861694   
        Illizi                  14169.601170             7.996085   
        Khenchela               10830.580811             4.852861   
        Laghouat                10084.528374             1.553661   
        Ouargla                 12379.121096            46.392685   
        Tamanrasset             15656.343103            11.531595   
        Tebessa                 10927.332730            13.398558   
Libya   Ghadamis                10515.471347            34.139752   
        Gharyan                 10818.727280            37.951196   
        Jufrah                  15113.240202            30.436329   
        Musrata                  8890.634530            47.423894   
Tunisia Gabes                   10074.150089            10.892591   
        Kebili                  11207.277369            20.871984   
        Tataouine               10215.633880            10.289599   
        Tozeur                  10794.142744            25.285805   

                     Average lcoe ($/kWh)  Pumping cost (M$)  \
country province                                               
Algeria Adrar                    0.612801          72.400425   
        Biskra                   0.565520          11.530187   
        Djelfa                   0.568539           1.763865   
        El Oued                  0.601037         121.106957   
        Ghardaia                 0.593385          49.763407   
        Illizi                   0.597699           4.758206   
        Khenchela                0.578422           2.935296   
        Laghouat                 0.551597           0.857584   
        Ouargla                  0.603499          28.026023   
        Tamanrasset              0.594153           6.868719   
        Tebessa                  0.606881           8.138726   
Libya   Ghadamis                 0.598376          20.486985   
        Gharyan                  0.815585          31.702263   
        Jufrah                   0.515105          15.675742   
        Musrata                  0.603778          28.765609   
Tunisia Gabes                    0.766064           8.335051   
        Kebili                   0.737514          15.235587   
        Ta